In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import random
from torch import nn
from glob import glob
from tqdm.auto import tqdm
from torchaudio import transforms as T
torch.cuda.is_available()

True

In [3]:
from maatool.data.feats_itdataset_v2 import FeatsIterableDatasetV2
from maatool.models.transformer import TransformerWithSinPos

In [4]:
import pytorch_lightning as pl 

In [5]:
import logging
import logging.config

def configure_logging(log_level):
    handlers =  {
            "maa": {
                "class": "logging.StreamHandler",
                "formatter": "maa_basic",
                "stream": "ext://sys.stdout",
            }
    }
    CONFIG = {
        "version": 1,
        "disable_existing_loggers": False,
        "formatters": {"maa_basic": {"format": '%(asctime)s %(name)s %(pathname)s:%(lineno)d - %(levelname)s - %(message)s'}},
        "handlers": handlers,
        "loggers": {"maa": {"handlers": handlers.keys(), "level": log_level}},
        "root": {"handlers": handlers.keys(), "level": log_level}
    }
    logging.config.dictConfig(CONFIG)
configure_logging("INFO")

In [6]:
torch.distributed.is_initialized()

False

In [7]:
from collections import defaultdict

In [8]:
def get_new_tgt(prev_tgt, hyp_logprobs, logits, topk=4):
    """
    prev_tgt - (T, N_hyp)
    hyp_logprobs - (N_hyp, )
    logits - (N_hyp, C)
    """
    assert len(prev_tgt.shape) == 2, f"{prev_tgt.shape=}"
    assert len(hyp_logprobs.shape) == 1, f"{hyp_logprobs.shape=}"
    assert len(logits.shape) == 2, f"{logits.shape=}"
    assert prev_tgt.shape[1] == hyp_logprobs.shape[0] == logits.shape[0], (
        f"{prev_tgt.shape=} {hyp_logprobs.shape=} {logits.shape=}"
    )
        
    nt_topk_logits, nt_topk_idx = logits.topk(k=topk, axis=-1)
    #print("nt_topk_idx", nt_topk_idx, nt_topk_idx.shape)
    # (N, K)
    next_tokens = nt_topk_idx.T.reshape(1, -1)
    #print("next_tokens", next_tokens, next_tokens.shape)
    # (1, N*(repeat k times)) 
    # (T, N*(repeat k times))  
    new_hyp_tgt = torch.concatenate([prev_tgt.repeat(1, topk), next_tokens], axis=0)
    #print(f"{new_hyp_tgt=}", new_hyp_tgt.shape)
    # (T+1, N*(repeat k times))
    new_scores = nt_topk_logits.T.reshape(-1)
    # N*(repeat k times)
    prew_scores = hyp_logprobs.repeat(topk)
    #print("prew_scores", prew_scores)
    # N*(repeat k times)
    new_hyp_logprob = prew_scores + new_scores
    #print("new_hyp_logprob", new_hyp_logprob)
    new_hyp_logprob, idx = new_hyp_logprob.topk(k=topk)
    #print(idx)
    new_hyps = new_hyp_tgt[:, idx]
    #print("new_hyps", new_hyps, new_hyp_logprob)
    # (T+1, N*k), (N,)
    return new_hyps, new_hyp_logprob
    

    
tgt, logits = get_new_tgt(torch.LongTensor([[1,]]), torch.tensor([-1.]), torch.tensor([[-3, -4, -7, -2, -5]]))
print(">>>>\n", tgt, logits)
get_new_tgt(tgt, logits, torch.tensor([[100,    110,  200], 
                                       [100,    110,  200], 
                                       [100,    110,  200], 
                                       [100,    110,  200]]), topk=2)

>>>>
 tensor([[1, 1, 1, 1],
        [3, 0, 1, 4]]) tensor([-3., -4., -5., -6.])


(tensor([[1, 1],
         [3, 0],
         [2, 2]]),
 tensor([197., 196.]))

In [9]:
def sep_ready_tgt(tgt, logprobs, eos_id=2):
    """
    tgt - (T, N)
    logprobs - (N,)
    """
    assert tgt.shape[1] == logprobs.shape[0], (
        f"{tgt.shape=} {logprobs.shape=}"
    )
    
    is_end_mask = ((tgt == eos_id).sum(axis=0) > 0)
    # (N,)
    #print(is_end_mask)
    ready_tgt = tgt[:, is_end_mask]
    ready_logprobs = logprobs[is_end_mask]
    
    ready_list = [(l.cpu().item(), t.cpu().tolist()) for l, t in zip(ready_logprobs, ready_tgt.T)]

    not_ready_tgt = tgt[:, ~is_end_mask]
    not_ready_logprobs = logprobs[~is_end_mask]
    assert not_ready_tgt.shape[1] == not_ready_logprobs.shape[0], (
        f"{not_ready_tgt.shape[1]=} {not_ready_logprobs.shape[0]=}"
    )
    return ready_list, not_ready_tgt, not_ready_logprobs


sep_ready_tgt(torch.LongTensor([[1, 1], [2, 3]]), torch.tensor([-1., -3]))

([(-1.0, [1, 2])],
 tensor([[1],
         [3]]),
 tensor([-3.]))

In [29]:
def set_random_seed(seed):
    if seed < 0:
        seed = seed_from_time()
    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
set_random_seed(42)

class SwipeTransformerRecognizer(pl.LightningModule):
    def __init__(self, backbone, learning_rate=1e-4, speed=42):
        super().__init__()
        self.save_hyperparameters(ignore=['backbone'])
        self.backbone = backbone
        self.ce_loss = nn.CrossEntropyLoss(ignore_index=0, reduction='mean')
#         self.spec_aug = torch.nn.Sequential(
#             #T.FrequencyMasking(freq_mask_param=24),
#             #T.TimeMasking(time_mask_param=30),
#             T.TimeMasking(time_mask_param=24), # last dim masking
#         )
        set_random_seed(speed)

    def forward(self, feats, **kwargs):
        # (T, N, E)
#         feats = feats.permute(1, 2, 0)
#         # (N, E, T)
#         feats = self.spec_aug(feats).permute(2, 0, 1)
#         if self.training:
#             #logging.info("Apply specaug")
#             feats = self.spec_aug(feats)
        # (T, N, E)
        return self.backbone(feats, **kwargs)
    
    def get_loss(self, batch):
        # batch - (Time, Batch, ...)
        feats = batch['feats']
        # (Time, Batch, num_feats)
        tgt = batch['targets'][:-1]
        tgt_key_padding_mask = batch['tgt_key_padding_mask'][:, 1:] 
        # (Batch, Seq-1)
        
        logits = self.forward(feats=feats, 
                              tgt=tgt, 
                              src_key_padding_mask=batch['src_key_padding_mask'], 
                              tgt_key_padding_mask=tgt_key_padding_mask) 
        # (Seq-1, Batch, C)
        S, N, C = logits.shape
        targets = batch['targets'][1:]
        # (Seq-1, Batch)
        # print("loss ", logits.shape, targets.shape)
        loss = self.ce_loss(logits.view(-1, C), targets.reshape(-1))
        
        return loss

        
    def training_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('train_loss', loss, on_epoch=True, prog_bar=True,  batch_size=len(batch['uids']))
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('valid_loss', loss, on_step=True,on_epoch=True, prog_bar=True, batch_size=len(batch['uids']))

    def test_step(self, batch, batch_idx):
        loss = self.get_loss(batch)
        self.log('test_loss', loss,  batch_size=len(batch['uids']))

    def configure_optimizers(self):
        # self.hparams available because we called self.save_hyperparameters()
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

    @staticmethod
    def add_model_specific_args(parent_parser):
        parser = ArgumentParser(parents=[parent_parser], add_help=False)
        parser.add_argument('--learning_rate', type=float, default=0.0001)
        return parser
    
    def predict_topk(self, dl, tokenizer, topk=4, bos_id=1, eos_id=2, max_out_len=26, device='cuda'):
        self.eval()
        utt2word= defaultdict(list)
        utt2logs = defaultdict(list)
        pbar = tqdm(dl)
        with torch.no_grad():
            for batch in pbar:
                batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
                memory = self.backbone.forward_encoder(batch['feats'], 
                                                  src_key_padding_mask=batch['src_key_padding_mask'])
                assert memory.shape[1] == 1, f"{memory.shape=}"
                # (SrcTime, Batch, E)
                tgt = torch.full(size=(1, 1), 
                                 fill_value=bos_id, 
                                 dtype=torch.long, 
                                 device=memory.device)
                hyp_logprobs = torch.zeros((1), device=memory.device)
                tgt_ready = []
                mkpm = batch['src_key_padding_mask']
                for l in range(max_out_len):
                    #print(f"{tgt.shape=}")
                    tgt_logits = self.backbone.forward_decoder(tgt, 
                                                        memory.repeat(1, tgt.shape[1], 1), 
                                                        memory_key_padding_mask=mkpm.repeat((tgt.shape[1], 1)))
                    tgt_logits = tgt_logits.log_softmax(dim=-1)

                    new_tgt, logprobs = get_new_tgt(tgt, hyp_logprobs, tgt_logits[-1], topk=topk)
                    ready, tgt, hyp_logprobs = sep_ready_tgt(new_tgt, logprobs)
                    tgt_ready.extend(ready)
                    if len(tgt_ready) >= topk:
                        break

                uid = batch['uids'][0]
                if len(tgt_ready) == 0:
                    logging.warning(f"tgt_ready is 0 for {uid}. {tgt.shape=}. Use all hyps as ready hyps")
                    tgt_ready = [(l.cpu().item(), t.cpu().tolist()) for l, t in zip(hyp_logprobs, tgt.T)]

                out_indices = []
                for logprob, indices in sorted(tgt_ready, reverse=True):
                    joined = tokenizer.decode(indices) #.split()[0]
                    utt2word[uid].append(joined)
                    utt2logs[uid].append(logprob)
                d = '|'+'|'.join(utt2word[uid]) + "|"
                pbar.set_description(f"{d}\t".ljust(40, '=')[:40], refresh=False)
        return utt2word, utt2logs

In [30]:
model = TransformerWithSinPos(feats_dim=37, num_tokens=500)
pl_module = SwipeTransformerRecognizer.load_from_checkpoint('exp/models/transformer_sc/lightning_logs/version_50424998/checkpoints/last.ckpt',backbone=model, map_location='cpu' )
#pl_module = SwipeTransformerRecognizer.load_from_checkpoint(
#    'exp/models/t_finetune_with_sa/lightning_logs/version_50448424/checkpoints/last-v1.ckpt',
#    backbone=model, 
#    map_location='cpu' )


PositionalEncoding shape is torch.Size([400, 1, 512])


In [14]:
val_ds = FeatsIterableDatasetV2([f"ark:data_feats/valid/feats.ark"], 
                             targets_rspecifier='ark:exp/bpe500/valid-text.int', 
                                shuffle=False,
                               bos_id=1, 
                               eos_id=2,
                               batch_first=False)
val_dataloader = torch.utils.data.DataLoader(val_ds, batch_size=1, collate_fn=val_ds.collate_pad)

2023-11-11 23:03:40,552 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:44 - INFO - Loading targets from ark:exp/bpe500/valid-text.int


Loading targets...: 0it [00:00, ?it/s]

In [13]:
train_ds = FeatsIterableDatasetV2([f"ark:{f}" for f in sorted(glob("data_feats/train/feats.*.ark"))],
                                  targets_rspecifier='ark:exp/bpe500/train-text.int.ark', 
                                  shuffle=True,
                                  bos_id=1, 
                                  eos_id=2, 
                                 batch_first=False)

train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=24, collate_fn=train_ds.collate_pad, 
                                                num_workers=8)

2023-11-11 10:56:31,307 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:44 - INFO - Loading targets from ark:exp/bpe500/train-text.int.ark


Loading targets...: 0it [00:00, ?it/s]

In [31]:
train_ds = FeatsIterableDatasetV2([f"ark:{f}" for f in sorted(glob("data_feats/suggestion_accepted/feats.*.ark"))],
                                  targets_rspecifier='ark:exp/bpe500/suggestion_accepted-text.int', 
                                  shuffle=True,
                                  bos_id=1, 
                                  eos_id=2, 
                                 batch_first=False)

train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=24, collate_fn=train_ds.collate_pad, 
                                                num_workers=8)

2023-11-11 23:36:33,439 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:44 - INFO - Loading targets from ark:exp/bpe500/suggestion_accepted-text.int


Loading targets...: 0it [00:00, ?it/s]

In [32]:
trainer = pl.Trainer(max_epochs=6, log_every_n_steps=400, reload_dataloaders_every_n_epochs=1,
                    default_root_dir='exp/models/t_finetune_with_sa',
                    callbacks=[pl.callbacks.TQDMProgressBar(refresh_rate=100),
                              pl.callbacks.ModelCheckpoint(every_n_train_steps=10000,
                                                          save_last=True)],
                    accumulate_grad_batches=4,
                    val_check_interval=20000)
                    #check_val_every_n_epoch=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [33]:
result = trainer.test(pl_module, val_dataloader)
print(result)
# 0.20462335646152496
# [{'test_loss': 2.9619081020355225}]
# v3.11.11 [{'test_loss': 0.6646422147750854}] /0.27 / 0.23

# [{'test_loss': 0.20462335646152496}]
# last 11.11 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:120: UserWarning: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

2023-11-11 23:37:58,703 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.20462335646152496    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.20462335646152496}]


In [ ]:
trainer.fit(pl_module, train_dataloader, val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                  | Params
---------------------------------------------------
0 | backbone | TransformerWithSinPos | 44.7 M
1 | ce_loss  | CrossEntropyLoss      | 0     
---------------------------------------------------
44.7 M    Trainable params
0         Non-trainable params
44.7 M    Total params
178.690   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Sanity Checking: 0it [00:00, ?it/s]

2023-11-11 23:39:19,443 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark


/mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/anaconda/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 96 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


2023-11-11 23:39:20,291 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.108.ark
2023-11-11 23:39:20,291 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.113.ark
2023-11-11 23:39:20,293 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.22.ark
2023-11-11 23:39:20,291 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.71.ark
2023-11-11 23:39:20,315 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_fe

...

2023-11-12 16:52:38,044 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark
2023-11-12 16:54:39,190 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.34.ark
2023-11-12 16:54:39,371 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.79.ark
2023-11-12 16:54:39,554 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.122.ark
2023-11-12 16:54:39,738 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_acc

Validation: 0it [00:00, ?it/s]

2023-11-12 17:38:08,479 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark
2023-11-12 17:40:27,546 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.128.ark
2023-11-12 17:40:27,729 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.28.ark
2023-11-12 17:40:27,802 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.21.ark
2023-11-12 17:40:27,918 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_acc

Validation: 0it [00:00, ?it/s]

2023-11-12 18:23:50,824 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/valid/feats.ark
2023-11-12 18:26:30,382 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.85.ark
2023-11-12 18:26:30,553 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.114.ark
2023-11-12 18:26:30,740 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_accepted/feats.65.ark
2023-11-12 18:26:30,842 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/suggestion_acc

In [ ]:
import sentencepiece as spm
import math
tokenizer = spm.SentencePieceProcessor('exp/bpe500/model.model')

In [ ]:
utt2words, utt2logs = pl_module.cuda().predict_topk(val_dataloader, tokenizer=tokenizer, topk=10, device='cuda')

In [ ]:
utt2words

In [ ]:
def accuracy(ref_u2w, hyp_u2w):
    corr = 0
    err = 0
    total = len(ref_u2w)
    for u, ref in tqdm(ref_u2w.items()):
        hyp = hyp_u2w[u].strip('-')
        if ref != hyp:
            print(ref, hyp)
            err +=1
        else:
            corr +=1
    a = corr/total
    print(f"{total=} {corr=} {err=}, accuracy: {a}")
    return a

with open('data_feats/valid/text') as f:
    valid_ref_u2w = {u:w for u, w in   map(str.split, f.readlines())}
    

In [ ]:
accuracy(valid_ref_u2w, {k:v[0] for k, v in utt2words.items()})
# v2.topk2 total=10000 corr=8429 err=1571, accuracy: 0.8429
# v2.topk5 total=10000 corr=8434 err=1566, accuracy: 0.8434
# v2.topk10 total=10000 corr=8388 err=1612, accuracy: 0.8388
# v3.topk10 total=10000 corr=8519 err=1481, accuracy: 0.8519  <--
# v3.11.11.topk10 total=10000 corr=8340 err=1660, accuracy: 0.834

In [26]:
with open('./data/voc.txt') as f:
    vocab = frozenset(s for s in map(str.strip, f.readlines()))

In [27]:
lv = {}
for k, v in utt2words.items():
    corr_w = None
    for w in v:
        if w in vocab:
            corr_w = w
            break
    if corr_w is None: 
        logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
        corr_w = '-'
    lv[k] = corr_w
accuracy(valid_ref_u2w, lv)
# v2.topk10 total=10000 corr=8542 err=1458, accuracy: 0.8542
# v3.topk10 total=10000 corr=8665 err=1335, accuracy: 0.8665
# v3.11.11.topk10 total=10000 corr=8429 err=1571, accuracy: 0.8429

2023-11-11 23:29:47,411 root /tmp/ipykernel_1307934/618118840.py:9 - WARNING - k='valid-320' doesn't have any vocab hyp. v=['агркина', 'агрикина', 'агруина', 'агресс', 'агрикона', 'агрикана', 'агпкина', 'агракина', 'агрессина', 'агрессив']
2023-11-11 23:29:47,412 root /tmp/ipykernel_1307934/618118840.py:9 - WARNING - k='valid-797' doesn't have any vocab hyp. v=['реабилизацию', 'реабилизовать', 'реабилизировать', 'реабилировать', 'реабилизация', 'реабилистировать', 'реабилизации', 'реабилистывать', 'реабилированная', 'реабилизированная', 'реабилизированный', 'реабилизированно']
2023-11-11 23:29:47,415 root /tmp/ipykernel_1307934/618118840.py:9 - WARNING - k='valid-4168' doesn't have any vocab hyp. v=['цистыр', 'цитва', 'цистова', 'цистыре', 'цитрамова', 'цистава', 'цистые', 'цатва', 'цитраства', 'цитраство']
2023-11-11 23:29:47,416 root /tmp/ipykernel_1307934/618118840.py:9 - WARNING - k='valid-4718' doesn't have any vocab hyp. v=['сверхностейство', 'сверхностественно', 'сверхностенно',

  0%|          | 0/10000 [00:00<?, ?it/s]

была быстро
рам нас
колывань кровь
шакалов шакалы
замазала запихала
воля волос
ура уля
баку бак
корень удали
купи купим
говорить говорит
водитель водителю
вечером вечер
фиолетовой фиолетовое
выехал выезжать
мы ты
черна черная
авто авито
вызовов вызвала
пробовал проблем
же жду
русскому русском
мазок мажор
не на
обувь обед
он бере
никогда никого
не неа
пойми пофиг
кар кал
прочел почему
же жду
кн кг
виде видела
не ок
прошу поза
дура будет
ск скок
агапкина 
дьявол добавь
ма мак
верон вероника
завтра звоню
два давай
тыс там
стоит строит
ощущение одной
выбил фабри
не ну
прошел пошел
можно может
глав глава
работы работаю
доллар дождусь
занимаешься зареветь
жень день
крепла крепко
приветик привет
мм ммм
выглядят выглядит
хо это
крот кот
потому поиск
мемы мены
сторону стону
кожа клас
день лет
привете приветик
ест есть
лежит делись
как кассе
но ер
дека девочка
не ну
сопли сдали
екб куб
же жду
прожить пожить
марин мари
подпрыгнул подростки
лай дай
носу номеру
проспект правильно
считал считала
але

0.8402

In [35]:
test_ds =  FeatsIterableDatasetV2([f"ark:data_feats/test/feats.ark"], shuffle=False, 
                                 bos_id=1, 
                                 eos_id=2, 
                                 batch_first=False)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=1, collate_fn=test_ds.collate_pad)
#test_u2w = predict(pl_module.backbone, test_dataloader)
test_u2w, test_u2l = pl_module.predict_topk(test_dataloader, tokenizer=tokenizer, topk=8)

0it [00:00, ?it/s]

2023-11-11 00:33:57,703 root /mnt/asr_hot/mitrofanov-aa/projects/chime7/chime7_stc_recipe/egs/it9/ya/maatool/data/feats_itdataset_v2.py:68 - INFO - Processing ark:data_feats/test/feats.ark


In [36]:
def limit_vocab(u2w, vocab=vocab):
    lv = {}
    for k, v in u2w.items():
        corr_w = []
        for w in v:
            if w in vocab:
                corr_w.append(w)
        if len(corr_w) == 0: 
            logging.warning(f"{k=} doesn't have any vocab hyp. {v=}")
            corr_w = ['-']
        lv[k] = corr_w
    return lv
test_lv = limit_vocab(test_u2w)

2023-11-11 00:38:42,715 root /tmp/ipykernel_3108189/1348474471.py:9 - WARNING - k='test-1319' doesn't have any vocab hyp. v=['госпориемка', 'госпоринска', 'гостроиемка', 'гостерика', 'госпитариемка', 'гостериемка', 'госпоринка', 'госпитарием', 'гостерикамка']
2023-11-11 00:38:42,715 root /tmp/ipykernel_3108189/1348474471.py:9 - WARNING - k='test-1360' doesn't have any vocab hyp. v=['барберина', 'барберика', 'барбекина', 'бараберина', 'барбериев', 'барбепина', 'барберины', 'барберинга']
2023-11-11 00:38:42,717 root /tmp/ipykernel_3108189/1348474471.py:9 - WARNING - k='test-1703' doesn't have any vocab hyp. v=['циклофен', 'цикоферон', 'цикофрон', 'циклоферн', 'циклофере', 'цикофре', 'циклоферо', 'циклоферов']
2023-11-11 00:38:42,718 root /tmp/ipykernel_3108189/1348474471.py:9 - WARNING - k='test-1765' doesn't have any vocab hyp. v=['шезтинг', 'шензион', 'щезтион', 'щезтики', 'щезтик', 'щезьмон', 'щезток', 'щезтин', 'щезтинг']
2023-11-11 00:38:42,719 root /tmp/ipykernel_3108189/1348474471

In [37]:
import pandas as pd

In [38]:
baseline_result = pd.read_csv('exp/models/ctc_trans/lightning_logs/version_50422251/test_submit.v1.csv', sep=',', names=['main', 'second', 'third', 'trash'])
baseline_result['uid'] = [f'test-{i}' for i in range(len(baseline_result))]
baseline_result.head()

,main,second,third,trash,uid
0,на,неа,на,ненка,test-0
1,что,часто,частого,чисто,test-1
2,опоздания,опозданиям,оприходования,опозданиями,test-2
3,сколько,сокольского,свердловского,скроено,test-3
4,дремать,дописать,донимать,дюрренматт,test-4


In [39]:
baseline_result['predict'] = baseline_result.uid.apply(lambda u: test_lv[u])
baseline_result.head()

,main,second,third,trash,uid,predict
0,на,неа,на,ненка,test-0,"[на, нас, нана, нам, неа, на-на]"
1,что,часто,частого,чисто,test-1,"[что, что-то, сто, чтоб, чисто, со, часто, чмок]"
2,опоздания,опозданиям,оприходования,опозданиями,test-2,"[опоздания, опоздание, опозданиям, опозданий, ..."
3,сколько,сокольского,свердловского,скроено,test-3,"[сколько, скольки, сколько, столько, только, с..."
4,дремать,дописать,донимать,дюрренматт,test-4,"[дремать, донимать, думать]"


In [40]:
rows = []

for i, row in baseline_result.iterrows():
    ps = row['predict']
    for p in [row['main'], row['second'], row['third'], row['trash']]:
        if p not in ps:
            ps.append(p)
    rows.append(ps[:4])
        
submission = pd.DataFrame(rows, columns=['main', 'second', 'third', 'trash'])
submission.head()

,main,second,third,trash
0,на,нас,нана,нам
1,что,что-то,сто,чтоб
2,опоздания,опоздание,опозданиям,опозданий
3,сколько,скольки,сколько,столько
4,дремать,донимать,думать,дописать


In [41]:
submission.to_csv("exp/models/transformer_sc/lightning_logs/version_50424998/test_submit.v5.csv", 
                  sep=',', header=False, index=False)